# Global AI Job Market & Salary Trends 2025

The dataset was obtained from the Kaggle website.

## Import required libraries and data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import gdown

In [2]:
# Get the dataset
# https://drive.google.com/file/d/1dsikUM2umj1sD19A_RXB-v9a2vPziXyb/view?usp=sharing
file_id = "1dsikUM2umj1sD19A_RXB-v9a2vPziXyb"
url = f"https://drive.google.com/uc?id={file_id}"
output = "job_market.csv"

gdown.download(url, output, quiet=False)

# Load the dataset
df = pd.read_csv(output)
df.head()

Downloading...
From: https://drive.google.com/uc?id=1dsikUM2umj1sD19A_RXB-v9a2vPziXyb
To: /content/job_market.csv
100%|██████████| 2.59M/2.59M [00:00<00:00, 43.4MB/s]


,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
0,AI00001,AI Research Scientist,90376,USD,SE,CT,China,M,China,50,"Tableau, PyTorch, Kubernetes, Linux, NLP",Bachelor,9,Automotive,2024-10-18,2024-11-07,1076,5.9,Smart Analytics
1,AI00002,AI Software Engineer,61895,USD,EN,CT,Canada,M,Ireland,100,"Deep Learning, AWS, Mathematics, Python, Docker",Master,1,Media,2024-11-20,2025-01-11,1268,5.2,TechCorp Inc
2,AI00003,AI Specialist,152626,USD,MI,FL,Switzerland,L,South Korea,0,"Kubernetes, Deep Learning, Java, Hadoop, NLP",Associate,2,Education,2025-03-18,2025-04-07,1974,9.4,Autonomous Tech
3,AI00004,NLP Engineer,80215,USD,SE,FL,India,M,India,50,"Scala, SQL, Linux, Python",PhD,7,Consulting,2024-12-23,2025-02-24,1345,8.6,Future Systems
4,AI00005,AI Consultant,54624,EUR,EN,PT,France,S,Singapore,100,"MLOps, Java, Tableau, Python",Master,0,Media,2025-04-15,2025-06-23,1989,6.6,Advanced Robotics


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   job_id                  15000 non-null  object 
 1   job_title               15000 non-null  object 
 2   salary_usd              15000 non-null  int64  
 3   salary_currency         15000 non-null  object 
 4   experience_level        15000 non-null  object 
 5   employment_type         15000 non-null  object 
 6   company_location        15000 non-null  object 
 7   company_size            15000 non-null  object 
 8   employee_residence      15000 non-null  object 
 9   remote_ratio            15000 non-null  int64  
 10  required_skills         15000 non-null  object 
 11  education_required      15000 non-null  object 
 12  years_experience        15000 non-null  int64  
 13  industry                15000 non-null  object 
 14  posting_date            15000 non-null

## Data Cleaning

- Check for missing values
- Convert columns to appropriate types
- Add colunms with shortcuts decode


| Column name | Initial type | Converted type |
|-------------|--------------|----------------|
| posting_date | string | datetime |
| application_deadline | string | datetime |
| required_skills | string | array |


In [4]:
def parce_array (df_string):
    try:
        return eval(df_string)
    except:
        return []

def parce_dates (df_string):
    try:
        return pd.to_datetime(df_string, format='%Y-%m-%d', errors='coerce')
    except Exception:
        return pd.Series([pd.NaT] * len(df_string))

def decode_column(df, df_column):
    mapping = {
        # employment_type
        'FT': 'Full-time',
        'PT': 'Part-time',
        'CT': 'Contract',
        'FL': 'Freelance',
        'IN': 'Internship',
        'TP': 'Temporary',

        # experience_level
        'EN': 'Entry',
        'MI': 'Mid',
        'SE': 'Senior',
        'EX': 'Executive'
    }
    new_column = f"{df_column}_full"
    df[new_column] = df[df_column].map(mapping).fillna('Other')
    return df

def clean_dataframe(df):
    df = df.copy()

    df['required_skills'] = df['required_skills'].apply(parce_array)
    df['posting_date'] = df['posting_date'].apply(parce_dates)
    df['application_deadline'] = df['application_deadline'].apply(parce_dates)

    df = decode_column(df, 'employment_type')
    df = decode_column(df, 'experience_level')

    return df

df = clean_dataframe(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   job_id                  15000 non-null  object        
 1   job_title               15000 non-null  object        
 2   salary_usd              15000 non-null  int64         
 3   salary_currency         15000 non-null  object        
 4   experience_level        15000 non-null  object        
 5   employment_type         15000 non-null  object        
 6   company_location        15000 non-null  object        
 7   company_size            15000 non-null  object        
 8   employee_residence      15000 non-null  object        
 9   remote_ratio            15000 non-null  int64         
 10  required_skills         15000 non-null  object        
 11  education_required      15000 non-null  object        
 12  years_experience        15000 non-null  int64 

## Export to Google Sheets
Export analysis results to Google Sheets using gspread.

In [5]:
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

spreadsheet = gc.create('JM_Analysis')
worksheet = spreadsheet.get_worksheet(0)

set_with_dataframe(worksheet, df)

spreadsheet.url

'https://docs.google.com/spreadsheets/d/1KKePTw799aorZAtmETxe7bE2t5KRS1uA5j7jLy3viBE'